<a href="https://colab.research.google.com/github/lillianphyo/mlopszoomcamp2024/blob/master/mlops_zoomcamp_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Forecasting AWS Cloud Expenses: Utilizing Billing Data for Price Projection

Install required packages

In [ ]:
!pip install numpy pandas matplotlib seaborn keras tensorflow

Q1  Downloading the data <br>

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

16
17
18
19

In [9]:
import pandas as pd
import pyarrow.parquet as pq

# Replace 'your_file.parquet' with the path to your Parquet file
file_path = '/content/yellow_tripdata_2023-01.parquet'

try:
    # Reading Parquet file into a DataFrame
    df = pd.read_parquet(file_path)
    print(df)
except Exception as e:
    print(f"Error reading Parquet file: {e}")


         VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0               2  2023-01-01 00:32:10   2023-01-01 00:40:36              1.0   
1               2  2023-01-01 00:55:08   2023-01-01 01:01:27              1.0   
2               2  2023-01-01 00:25:04   2023-01-01 00:37:49              1.0   
3               1  2023-01-01 00:03:48   2023-01-01 00:13:25              0.0   
4               2  2023-01-01 00:10:29   2023-01-01 00:21:19              1.0   
...           ...                  ...                   ...              ...   
3066761         2  2023-01-31 23:58:34   2023-02-01 00:12:33              NaN   
3066762         2  2023-01-31 23:31:09   2023-01-31 23:50:36              NaN   
3066763         2  2023-01-31 23:01:05   2023-01-31 23:25:36              NaN   
3066764         2  2023-01-31 23:40:00   2023-01-31 23:53:00              NaN   
3066765         2  2023-01-31 23:07:32   2023-01-31 23:21:56              NaN   

         trip_distance  Rat

Q2 Computing duration <br>
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

32.59
42.59
52.59
62.59

In [11]:
import pandas as pd

# URL for the January 2023 Yellow Taxi Trip Records (you need to update this with the actual URL)
url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet'

# Read the dataset from the URL
try:
    df = pd.read_parquet(url)

    # Ensure the pickup and dropoff datetime columns are in datetime format
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

    # Compute the trip duration in minutes
    df['trip_duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

    # Calculate the standard deviation of the trip durations
    std_dev = df['trip_duration'].std()
    print(f"The standard deviation of the trip durations in January 2023 is: {std_dev:.2f} minutes")
except Exception as e:
    print(f"An error occurred: {e}")


The standard deviation of the trip durations in January 2023 is: 42.59 minutes


Q3 Dropping outliers <br>
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

90%
92%
95%
98%

In [12]:
import pandas as pd

# URL for the January 2023 Yellow Taxi Trip Records (you need to update this with the actual URL)
url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet'

# Read the dataset from the URL
try:
    df = pd.read_parquet(url)

    # Ensure the pickup and dropoff datetime columns are in datetime format
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

    # Compute the trip duration in minutes
    df['trip_duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

    # Calculate the original number of records
    original_count = len(df)

    # Filter records to keep durations between 1 and 60 minutes (inclusive)
    filtered_df = df[(df['trip_duration'] >= 1) & (df['trip_duration'] <= 60)]

    # Calculate the filtered number of records
    filtered_count = len(filtered_df)

    # Calculate the fraction of records remaining
    fraction_remaining = (filtered_count / original_count) * 100
    print(f"The fraction of records remaining after removing outliers is: {fraction_remaining:.2f}%")
except Exception as e:
    print(f"An error occurred: {e}")


The fraction of records remaining after removing outliers is: 98.12%


Q4 One-hot encoding <br>
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

2
155
345
515
715

In [13]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer

# URL for the January 2023 Yellow Taxi Trip Records (you need to update this with the actual URL)
url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet'

# Read the dataset from the URL
try:
    df = pd.read_parquet(url)

    # Ensure the pickup and dropoff datetime columns are in datetime format
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

    # Compute the trip duration in minutes
    df['trip_duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

    # Filter records to keep durations between 1 and 60 minutes (inclusive)
    df = df[(df['trip_duration'] >= 1) & (df['trip_duration'] <= 60)]

    # Convert location IDs to strings
    df['PULocationID'] = df['PULocationID'].astype(str)
    df['DOLocationID'] = df['DOLocationID'].astype(str)

    # Create a list of dictionaries
    data_dicts = df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

    # Fit a dictionary vectorizer
    vec = DictVectorizer()
    feature_matrix = vec.fit_transform(data_dicts)

    # Get the dimensionality of the feature matrix
    num_columns = feature_matrix.shape[1]
    print(f"The dimensionality of the feature matrix is: {num_columns} columns")
except Exception as e:
    print(f"An error occurred: {e}")


The dimensionality of the feature matrix is: 515 columns



Q5 Training a model <br>
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data
What's the RMSE on train?

3.64
7.64
11.64
16.64

In [14]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# URL for the January 2023 Yellow Taxi Trip Records (you need to update this with the actual URL)
url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet'

# Read the dataset from the URL
try:
    df = pd.read_parquet(url)

    # Ensure the pickup and dropoff datetime columns are in datetime format
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

    # Compute the trip duration in minutes
    df['trip_duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

    # Filter records to keep durations between 1 and 60 minutes (inclusive)
    df = df[(df['trip_duration'] >= 1) & (df['trip_duration'] <= 60)]

    # Convert location IDs to strings
    df['PULocationID'] = df['PULocationID'].astype(str)
    df['DOLocationID'] = df['DOLocationID'].astype(str)

    # Create a list of dictionaries
    data_dicts = df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

    # Fit a dictionary vectorizer
    vec = DictVectorizer()
    X = vec.fit_transform(data_dicts)

    # Get the response variable
    y = df['trip_duration'].values

    # Train a linear regression model
    model = LinearRegression()
    model.fit(X, y)

    # Predict on the training data
    y_pred = model.predict(X)

    # Calculate the RMSE
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    print(f"The RMSE on the training data is: {rmse:.2f}")
except Exception as e:
    print(f"An error occurred: {e}")


The RMSE on the training data is: 7.65


Q6 Evaluating the model <br>
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

3.81
7.81
11.81
16.81

In [15]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Function to preprocess and filter the dataset
def preprocess_data(url):
    df = pd.read_parquet(url)
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
    df['trip_duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
    df = df[(df['trip_duration'] >= 1) & (df['trip_duration'] <= 60)]
    df['PULocationID'] = df['PULocationID'].astype(str)
    df['DOLocationID'] = df['DOLocationID'].astype(str)
    return df

# URL for the January 2023 Yellow Taxi Trip Records
january_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet'

# Preprocess January data
df_january = preprocess_data(january_url)

# Create a list of dictionaries and fit the dictionary vectorizer
data_dicts_january = df_january[['PULocationID', 'DOLocationID']].to_dict(orient='records')
vec = DictVectorizer()
X_january = vec.fit_transform(data_dicts_january)
y_january = df_january['trip_duration'].values

# Train the linear regression model
model = LinearRegression()
model.fit(X_january, y_january)

# URL for the February 2023 Yellow Taxi Trip Records
february_url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet'

# Preprocess February data
df_february = preprocess_data(february_url)

# Transform February data using the same dictionary vectorizer
data_dicts_february = df_february[['PULocationID', 'DOLocationID']].to_dict(orient='records')
X_february = vec.transform(data_dicts_february)
y_february = df_february['trip_duration'].values

# Predict on the validation data
y_pred_february = model.predict(X_february)

# Calculate the RMSE on the validation data
rmse_february = np.sqrt(mean_squared_error(y_february, y_pred_february))
print(f"The RMSE on the validation data is: {rmse_february:.2f}")


The RMSE on the validation data is: 7.81
